In [1]:
def read_data(input_file):
    """
    Read data from stdin
    """
    with open(input_file, 'r') as f:
        lines = f.readlines()
        n, m = list(map(int, lines[0].split()))
        d = list(map(int, lines[1].split()))
        assert len(d) == n, "Number of classes does not match"
        c = list(map(int, lines[2].split()))
        assert len(c) == m, "Number of rooms does not match"
        k = int(lines[3])
        conflicts = []
        for i in range(4, 4 + k):
            conflicts.append(list(map(int, lines[i].split())))
    return n, m, d, c, conflicts


def build_conflict_table(n, conflicts):
    """
    Build a conflict table so we can check if two classes are conflicting in O(1)

    conflict_table[(i, j)] = 1 if class i and class j are conflicting
    """
    conflict_table = dict()
    for i, j in conflicts:
        conflict_table[(i, j)] = 1
        conflict_table[(j, i)] = 1

    for i in range(1, n + 1):
        conflict_table[(i, i)] = 1
        
    return conflict_table
                      

def schedule_exams(n, m, d, c, conflict_table):
    global best_objective, best_schedule_room, best_schedule_section

    scheduled_room = [0 for _ in range(n)]
    schedule_section = [0 for _ in range(n)]
    best_schedule_room = []
    best_schedule_section = []
    best_objective = (n - 1) // 4 + 2

    def is_valid(subject_index, section, room):
        # branch and bound
        if section // 4 + 1 >= best_objective:
            return False

        # check capacity
        if d[subject_index] > c[room]:
            return False
        
        # check room is not used at the same time
        for subject in range(subject_index):
            if scheduled_room[subject] == room and schedule_section[subject] == section:
                return False
            
        # check conflict
        for subject in range(subject_index):
            if conflict_table.get((subject + 1, subject_index + 1), 0) == 1:
                if schedule_section[subject] == section:
                    return False
                
        return True

    def solution_found():
        global best_objective, best_schedule_room, best_schedule_section
        max_section = max(schedule_section)
        if max_section < best_objective:
            best_objective = max_section // 4 + 1
            best_schedule_room = [room for room in scheduled_room]
            best_schedule_section = [section % 4 for section in schedule_section]


    def backtrack(subject_index):
        for section in range(n):
            for room in range(m):
                if is_valid(subject_index, section, room):
                    scheduled_room[subject_index] = room
                    schedule_section[subject_index] = section
                    
                    if subject_index == n - 1:
                        solution_found()
                    else:
                        backtrack(subject_index + 1)

    backtrack(0)

    final_answer = []
    for i in range(n):
        final_answer.append((i + 1, best_schedule_section[i] + 1, best_schedule_room[i] + 1))

    return final_answer, best_objective



class BackTrack:
    def __init__(self) -> None:
        pass    
    
    def solve(self, input_file: str):
        n, m, d, c, conflicts = read_data(input_file)
        conflict_table = build_conflict_table(n, conflicts)

        final_answer, best_objective = schedule_exams(n, m, d, c, conflict_table)

        print(f"Best objective: {best_objective}")

        return best_objective, n, m, c

In [5]:
import os
import time
import pandas as pd

test_cases = []
num_subjects = []
num_rooms = []
times = []
objective_values = []

solver = BackTrack()

for test_case in [ 'data_10_2_(0).txt', 'data_10_2_(1).txt', 'data_10_2_(2).txt', 
                  'data_16_3_(0).txt',  'data_16_3_(1).txt', 'data_16_3_(2).txt',
                  'data_20_4_(0).txt', 'data_20_4_(1).txt', 'data_20_4_(2).txt',
                  ]:
    print(f"Start solving {test_case}")
    start_time = time.time()
    ans, n, m, c = solver.solve("../data/" + test_case)
    end_time = time.time()
    test_cases.append(test_case)
    num_subjects.append(n)
    num_rooms.append(m)
    times.append(end_time - start_time)
    objective_values.append(ans)
    print(f"Finished solving {test_case}")
    print(f"Time elapsed: {end_time - start_time}")

result = pd.DataFrame({
    "test_case": test_cases,
    "num_subjects": num_subjects,
    "num_rooms": num_rooms,
    "time": times,
    "objective_value": objective_values
})

Start solving data_10_2_(0).txt


KeyboardInterrupt: 

In [ ]:
result

,test_case,num_subjects,num_rooms,num_conflicts,time,objective_value
0,data_10_2_(0).txt,10,2,10,0.005228,2
1,data_10_2_(1).txt,10,2,10,7.091517,3
2,data_10_2_(2).txt,10,2,10,0.000000,2
3,data_16_3_(0).txt,16,3,16,130.219400,2
4,data_16_3_(1).txt,16,3,16,122.805888,3
5,data_16_3_(2).txt,16,3,16,79.125592,2
6,data_20_4_(0).txt,20,4,20,0.753692,2
7,data_20_4_(1).txt,20,4,20,829.579995,2
8,data_20_4_(2).txt,20,4,20,43.922294,2


In [ ]:
result.to_csv("backtrack.csv", index=False)